This is adapted from the MIRI imaging JWST pipeline notebook

In [1]:
import os
import glob
import jwst
import crds

from pathlib import Path

import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from astroquery.mast import Observations
from astropy.table import Table
from astropy.io import fits

import matplotlib.pyplot as plt

from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# ------------------

import jwst
import crds

from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline

# JWST pipeline utilities
from jwst import datamodels
from jwst.datamodels import ImageModel
from jwst.associations import asn_from_list as afl  # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase  # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base  # Definition of a Lvl3 association file

# Echo pipeline version and CRDS context in use
print(f"JWST Calibration Pipeline Version: {jwst.__version__}")
print(f"Using CRDS Context: {crds.get_context_name('jwst')}")

import warnings
warnings.filterwarnings("ignore")

JWST Calibration Pipeline Version: 1.19.1


CRDS - INFO -  Calibration SW Found: jwst 1.19.1 (/home/users/jlu69/miniconda3/envs/jurassic/lib/python3.12/site-packages/jwst-1.19.1.dist-info)


Using CRDS Context: jwst_1413.pmap


In [2]:
user_home_dir = os.path.expanduser('~')
data_dir = os.path.join('/home/phys/astronomy/jlu69/Masters/jurassic/', 'pipeline_data')
download_dir = data_dir
sci_dir = os.path.join(data_dir, 'Obs')
bg_dir = ''
dodet1 = True  # calwebb_detector1
dodet1bg = True  # calwebb_detector1

In [3]:
# Check whether the local CRDS cache directory has been set.
# If not, set it to the user home directory
if (os.getenv('CRDS_PATH') is None):
    os.environ['CRDS_PATH'] = os.path.join(os.path.expanduser('~'), 'crds')

# Check whether the CRDS server URL has been set.  If not, set it.
if (os.getenv('CRDS_SERVER_URL') is None):
    os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

# Echo CRDS path in use
print(f"CRDS local filepath: {os.environ['CRDS_PATH']}")
print(f"CRDS file server: {os.environ['CRDS_SERVER_URL']}")

CRDS local filepath: /home/users/jlu69/crds_cache
CRDS file server: https://jwst-crds.stsci.edu


In [13]:
# Define output subdirectories to keep science data products organized
uncal_dir = os.path.join(sci_dir, 'uncal/', 'trappist-1')  # Uncalibrated pipeline inputs should be here
det1_dir = os.path.join(sci_dir, 'stage1/', 'trappist-1')  # calwebb_detector1 pipeline outputs will go here

# Output subdirectories to keep background data products organized
uncal_bgdir = os.path.join(bg_dir, 'uncal')  # Uncalibrated pipeline inputs should be here
det1_bgdir = os.path.join(bg_dir, 'stage1')  # calwebb_detector1 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not
# create them
if not os.path.exists(det1_dir):
    os.makedirs(det1_dir)

if ((bg_dir != '') & (not os.path.exists(det1_bgdir))):
    os.makedirs(det1_bgdir)

In [14]:
# Set up a dictionary to define how the Detector1 pipeline should be configured

# Boilerplate dictionary setup
det1dict = {}
det1dict['group_scale'], det1dict['dq_init'], det1dict['emicorr'] = {}, {}, {}
det1dict['saturation'], det1dict['firstframe'], det1dict['lastframe'] = {}, {}, {}
det1dict['reset'], det1dict['linearity'], det1dict['rscd'] = {}, {}, {}
det1dict['dark_current'], det1dict['refpix'], det1dict['jump'] = {}, {}, {}
det1dict['ramp_fit'], det1dict['gain_scale'] = {}, {}

# Overrides for whether or not certain steps should be skipped
det1dict['ramp_fit']['skip'] = True

# Overrides for various reference files

# Turn on multi-core processing (off by default).  Choose what fraction of cores to use (quarter, half, or all)
det1dict['jump']['maximum_cores'] = 'quarter'

In [15]:
uncal_files = sorted(glob.glob(os.path.join(uncal_dir, '*_uncal.fits')))
uncal_bgfiles = sorted(glob.glob(os.path.join(uncal_bgdir, '*_uncal.fits')))

print('Found ' + str(len(uncal_files)) + ' science input files')
print('Found ' + str(len(uncal_bgfiles)) + ' background input files')

Found 119 science input files
Found 0 background input files


In [16]:
if dodet1:
    # print file name
    print(uncal_files[0])

    # Open file as JWST datamodel
    examine = datamodels.open(uncal_files[0])

    # Print out exposure info
    print(f"Instrument: {examine.meta.instrument.name}")
    print(f"Filter: {examine.meta.instrument.filter}")
    print(f"Number of integrations: {examine.meta.exposure.nints}")
    print(f"Number of groups: {examine.meta.exposure.ngroups}")
    print(f"Readout pattern: {examine.meta.exposure.readpatt}")
    print(f"Dither position number: {examine.meta.dither.position_number}")
    print("\n")

/home/phys/astronomy/jlu69/Masters/jurassic/pipeline_data/Obs/uncal/trappist-1/jw01177007001_03101_00001-seg001_mirimage_uncal.fits
Instrument: MIRI
Filter: F1500W
Number of integrations: 377
Number of groups: 14
Readout pattern: FASTR1
Dither position number: 1




In [17]:
if dodet1:
    for uncal in uncal_files:
        rate_result = Detector1Pipeline.call(uncal, output_dir=det1_dir, steps=det1dict,save_calibrated_ramp=True,save_results=True,)
else:
    print('Skipping Detector1 processing')

2025-11-19 11:33:48,153 - stpipe - INFO - PARS-EMICORRSTEP parameters found: /home/users/jlu69/crds_cache/references/jwst/miri/jwst_miri_pars-emicorrstep_0003.asdf
2025-11-19 11:33:48,164 - stpipe - INFO - PARS-DARKCURRENTSTEP parameters found: /home/users/jlu69/crds_cache/references/jwst/miri/jwst_miri_pars-darkcurrentstep_0001.asdf
2025-11-19 11:33:48,172 - stpipe - INFO - PARS-JUMPSTEP parameters found: /home/users/jlu69/crds_cache/references/jwst/miri/jwst_miri_pars-jumpstep_0005.asdf
2025-11-19 11:33:48,180 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /home/users/jlu69/crds_cache/references/jwst/miri/jwst_miri_pars-detector1pipeline_0006.asdf
2025-11-19 11:33:48,192 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2025-11-19 11:33:48,192 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2025-11-19 11:33:48,193 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2025-11-19 11:33:48,194 - stpipe